---
layout: post
title:  "Ch02.Prompt"
date:   2015-06-26 02:00:00 +0700
categories: [LLM]
---

<script type="text/x-mathjax-config">
MathJax.Hub.Config({tex2jax: {inlineMath: [['$','$'], ['\\(','\\)']]}});
</script>
<script type="text/javascript" src="https://cdn.mathjax.org/mathjax/latest/MathJax.js?config=TeX-MML-AM_CHTML">
</script>

### 참조 사이트.
해당 Post는 <a href="https://wikidocs.net/book/14314">LangChain 위키독스</a>에 나와있는 예제와 흐름을 파악하는 용도 입니다.  

### Prompt

LLM의 Input으로 들어가게 되는 Prompt에 대해 자세히 알아본다.

Prompt란 아래와 같이 정의하고 있다.

>프롬프트 단계는 검색기에서 검색된 문서들을 바탕으로 **언어 모델이 사용할 질문이나 명령을 생성하는 과정**입니다. 이 단계는 **검색된 정보를 바탕**으로 최종 사용자의 질문에 가장 잘 대응할 수 있는 응답을 생성하기 위해 필수적인 단계입니다.

Prompt는 아래와 같은 필요성을 가지고 있다.
- 문맥(Context) 설정: LLM에 추가적인 정보를 전달함으로서 할루시네이션을 줄이는 역할로 사용 가능하다.
- 정보 통합: 서로 다른 여러 정보를 하나로 합쳐 정보를 전달 할 수 있다.
- 응답 품질 향상: RAG에서 많이 사용되는 방법으로서, 도움이 되는 Context를 선택하여 전달 할 수 있다.

기본적인 RAG에서 많이 사용되는 구조는 아래와 같다.
- Instruction: 지시사항으로서, LLM에 어떤 역할로서 답변을 얻어낼지 지정한다.
- Question: 사용자의 질문을 전달한다.
- Context: RAG로서 검색된 정보를 전달한다. 답변생성에 있어서 할루시네이션을 줄이는 역할을 한다.

기본적인 RAG의 Prompt를 확인하면 아래와 같다.

```python
당신은 질문-답변(Question-Answer) Task 를 수행한는 AI 어시스턴트 입니다.
검색된 문맥(context)를 사용하여 질문(question)에 답하세요. 
만약, 문맥(context) 으로부터 답을 찾을 수 없다면 '모른다' 고 말하세요. 
한국어로 대답하세요.

#Question: 
{이곳에 사용자가 입력한 질문이 삽입됩니다}

#Context: 
{이곳에 검색된 정보가 삽입됩니다}
```

### Setting

#### Import Library

In [1]:
import yaml
from typing import List
from pathlib import Path
from datetime import datetime

from langchain_openai import ChatOpenAI
from langchain.embeddings.base import Embeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import HumanMessage, AIMessage
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder, load_prompt
from langchain_core.output_parsers import StrOutputParser

#### Model 선언

In [2]:
model = ChatOpenAI(
    base_url="http://localhost:8000/v1",
    api_key="EMPTY",
    model="Qwen/Qwen3-4B"
)

#### 사용할 Prompt 저장.

사용할 Prompt를 미리 지정하여 저장한다.

Appendix: OpenAI에서 제공하는 default prompt가 있으나, 실제 해당 값과 동일하게 저장하고 불러와서 확인하다.

fruit_color.yaml

In [3]:
prompt_data = {
    "_type": "prompt",
    "input_variables": ["fruit"],
    "template": "What color is a {fruit}?"
}

Path("prompts").mkdir(exist_ok=True)
with open("./data/fruit_color.yaml", "w", encoding="utf-8") as f:
    yaml.dump(prompt_data, f, allow_unicode=True)

load_prompt로 불러와서 확인

In [4]:
prompt = load_prompt("./data/fruit_color.yaml")
print(prompt)

input_variables=['fruit'] input_types={} partial_variables={} template='What color is a {fruit}?'


실제 load_prompt를 확인하게 되면, 아래와 같이 type이 **langchain_core.prompts.prompt.PromptTemplate**인 것을 확인할 수 있다.  
위와 같이 바뀌는 이유는 LangChain인 내부에서 _type을 확인하고 자동으로 Mapping하여 가져오기 때문이다.

```python
if _type == "prompt":
    return PromptTemplate(...)
```

In [5]:
type(prompt)

langchain_core.prompts.prompt.PromptTemplate

### Prompt Template

#### 방법 1. from_template() 메소드를 사용하여 PrompteTemplate 객체를 생성

In [6]:
# template 정의. {country}는 변수로, 이후에 값이 들어갈 자리를 의미
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)
print(type(prompt))
print(prompt)

# chain 생성
chain = prompt | model

# country 변수에 입력된 값이 자동으로 치환되어 수행됨
chain.invoke("대한민국").content.split('</think>\n\n')[1]

<class 'langchain_core.prompts.prompt.PromptTemplate'>
input_variables=['country'] input_types={} partial_variables={} template='{country}의 수도는 어디인가요?'


'대한민국의 수도는 **서울**입니다.  \n서울은 한국의 정치, 경제, 문화 중심지로, 국가의 주요 행정 기관과 국제적인 중심 도시로 알려져 있습니다.  \n역사적으로 서울은 한국의 중심 도시로 자리 잡고 있으며, 일본의 식민지 시대에는 수도로 사용되었고, 1945년 해방 후 다시 서울이 수도로 정해졌습니다.  \n다른 주요 도시(부산, 인천 등)는 서울과는 구분되는 대도시이지만, 수도는 서울입니다.'

#### 방법 2. PromptTemplate 객체 생성과 동시에 prompt 생성

input_variables를 사용하여, PromptTemplate를 생성한다.

**appendix: number of input_variables**: input_variables가 1개인 경우에는 invoke의 값이 자동으로 귿어가게 된다.  
하지만, 2개 이상인 경우에는 dictionary 형태로 전달해야 LLM에 Input으로 사용할 수 있다.

input_variables가 1개인 경우.

In [7]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country"],
)

print(type(prompt))
print(prompt)

# chain 생성
chain = prompt | model

# country 변수에 입력된 값이 자동으로 치환되어 수행됨
chain.invoke("대한민국").content.split('</think>\n\n')[1]

<class 'langchain_core.prompts.prompt.PromptTemplate'>
input_variables=['country'] input_types={} partial_variables={} template='{country}의 수도는 어디인가요?'


'대한민국의 수도는 **서울**입니다.  \n서울은 한국의 정치, 경제, 문화 중심지로, 국가의 주요 기관(국회, 대통령 집안, 행정부 등)이 위치해 있습니다.  \n또한 서울은 역사적, 문화적 유산이 풍부한 도시로, 세계적인 관광지로도 유명합니다.  \n다른 대도시인 부산, 인천 등과는 달리, 서울은 대한민국의 정부 기관이 집중된 수도 도시입니다.'

input_variables가 2개인 경우

In [8]:
prompt = PromptTemplate(
    template="{country1}과 {country2}의 수도는 각각 어디인가요?",
    input_variables=["country1", "country2"],
)

# chain 생성
chain = prompt | model

# country 변수에 입력된 값이 자동으로 치환되어 수행됨
chain.invoke("대한민국").content.split('</think>\n\n')[1]

TypeError: Expected mapping type as input to PromptTemplate. Received <class 'str'>.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT 

In [9]:
chain.invoke({"country1": "대한민국", "country2": "일본"}).content.split('</think>\n\n')[1]

'대한민국의 수도는 **서울**이고, 일본의 수도는 **东京(도쿄)**입니다.  \n\n- **서울(Seoul)**: 대한민국의 정치, 경제, 문화 중심지로, 세계적인 도시로 유명합니다. 역사적인 장소인 경복궁과 현대적인 빌딩들이 어우러진 도시입니다.  \n- **도쿄(Tokyo)**: 일본의 정치, 경제, 문화 중심지로, 세계적인 재무 중심지로 자리잡고 있습니다. 역사적 유적지인 일본 왕실 궁전과 현대적인 도시 풍경이 조화롭게 어우러집니다.  \n\n두 도시 모두 각국의 중심지로서 중요한 역할을 합니다.'

partital_variables를 사용하여, dictionary형태로 input value와 default value를 적용할 수 있다.

In [10]:
# template 정의
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "미국"  # dictionary 형태로 partial_variables를 전달
    },
)

print(type(prompt))
print(prompt)

# chain 생성
chain = prompt | model

# country1 변수에 입력된 값이 자동으로 치환되어 수행됨 (input_variables가 1개 이므로.)
chain.invoke("대한민국").content.split('</think>\n\n')[1]

<class 'langchain_core.prompts.prompt.PromptTemplate'>
input_variables=['country1'] input_types={} partial_variables={'country2': '미국'} template='{country1}과 {country2}의 수도는 각각 어디인가요?'


'대한민국의 수도는 **서울**입니다.  \n미국의 수도는 **워싱턴 D.C.**(Washington, D.C.)입니다.  \n\n### 추가 정보:\n- **서울**은 대한민국의 정치, 경제, 문화 중심지로, 1948년 이후로 수도로 지정되어 있습니다.  \n- **워싱턴 D.C.**는 미국의 정부 기관(미국 대통령 주거지인 백악관, 상원 및 하원, Supreme Court 등)이 위치한 연방 구역으로, 주요 도시가 아닌 공식적인 수도입니다.  \n\n이 두 도시는 각국의 정치적 중심지로 중요한 역할을 합니다.'

**partitial variables**: partital_varirables에는 함수(Callable)의 값을 넣을 수 있지만, input_variables에는 함수는 안됩니다. 반드시 입력값으로 직접 값을 전달하여야 합니다.

partital variables에 함수값을 받는 대표적인 예시는 날짜를 반환하는 함수로서 사용법은 아래와 같습니다.

In [11]:
# 날짜를 반환하는 함수 정의
def get_today():
    return datetime.now().strftime("%B %d")

# prompt 정의
prompt = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.",
    input_variables=["n"],
    partial_variables={
        "today": get_today  # 함수의 return값으로서 partial_variables값 전달.
    },
)

# 결과 확인
chain = prompt | model
chain.invoke(3).content.split('</think>\n\n')[1]

'June 28일에 생일을 맞이한 유명인 3명은 다음과 같습니다. 각자의 생년월일을 기재하였습니다:\n\n1. **Sir Peter Ustinov (1928년 6월 28일 - 2004년 9월 28일)**  \n   - 영국의 배우, 풍자극작가, 정치인. "The Ladykillers" 등의 영화에서 유명한 인물로, 1980년대에 BBC에서 라디오 연기자로 활동했으며, 1990년대에는 정치적 라디오 프로그램을 진행했습니다.\n\n2. **Terry Pratchett (1948년 6월 28일 - 2015년 6월 12일)**  \n   - 영국의 소설가로, "Discworld" 시리즈를 기반으로 한 판타지 소설을 쓴 인물. "The Color of Magic"와 "Night Watch" 등이 유명합니다. 1990년대에 "The Hitchhiker\'s Guide to the Galaxy" 시리즈를 집필했으며, 2005년에 "Good Omens"을 공동 집필했습니다.\n\n3. **Lance Armstrong (1971년 6월 28일 - 현재)**  \n   - 미국의 자전거 선수로, 7회 토리에 챔피언십 우승자로 유명합니다. 1999년부터 2005년까지 7회 우승을 차지했지만, 2013년에 음주 및 약물 사용 혐의로 2005년 우승을 무효화당했습니다. 자전서 "The Tour"를 저술했습니다.\n\n이 외에도 일부 인물은 생일이 June 28일이지만, 그들의 유명도나 영향력이 상대적으로 낮은 경우가 있습니다. 위의 목록은 주로 인류 역사상 큰 영향을 미친 인물들을 중심으로 구성했습니다.'

함수값으로서 선언하여도, 아래와 같이 String으로 다시 값을 치환하여서 넣을 수 있다.

In [12]:
chain.invoke({"today": "Jan 02", "n": 3}).content.split('</think>\n\n')[1]

'지난 1월 2일에 태어난 유명인 3명을 아래에 소개합니다. 생년월일은 정확한 정보를 기반으로 제공되었으나, 일부 인물의 출생일이 오류가 있을 수 있으니 참고하시기 바랍니다.\n\n1. **스티븐 스皮尔버그 (Steven Spielberg)**  \n   - **생년월일**: 1946년 1월 2일  \n   - **소속 분야**: 영화 감독, 제작자  \n   - **특징**: 세계적으로 유명한 영화 감독으로 《인셉션》, 《조지아의 여름》 등 다양한 작품을 제작한 바 있다.\n\n2. **존 레전드 (John Legend)**  \n   - **생년월일**: 1987년 1월 2일  \n   - **소속 분야**: 가수, 작곡가  \n   - **특징**: 블랙 프라이데이 쇼를 비롯한 다양한 공연에서 활동하며, 음악적 재능을 인정받은 인물.\n\n3. **조지 루카스 (George Lucas)**  \n   - **생년월일**: 1944년 1월 2일  \n   - **소속 분야**: 영화 감독, 제작자, 시나리오작가  \n   - **특징**: 《스타워즈》 시리즈의 창작자로 영화 산업에 큰 영향을 미친 인물.\n\n**참고**: 일부 인물의 출생일이 오류가 있을 수 있으므로, 정확한 정보는 공식적인 출처를 확인하시기 바랍니다.'

#### 방법 3. File로서 값을 가져오는 방법.

LangChain에서 지원하는 PromptTemplate로서 file을 읽고 만드는 법은, 아래와 같이 load_prompt를 사용하는 방법이다.

In [13]:
prompt = load_prompt('./data/fruit_color.yaml')
chain = prompt | model
chain.invoke("사과").content.split('</think>\n\n')[1]

"A 사과 (apple) can come in various colors depending on the variety. The most common colors are **red**, **green**, and **yellow**, but other colors like **purple** or **orange** are also possible. For example:  \n- **Red apples** (e.g., Red Delicious)  \n- **Green apples** (e.g., Granny Smith)  \n- **Yellow apples** (e.g., Honeycrisp)  \n\nThe color is determined by the apple's species and growing conditions. So, while red is the most typical answer, apples can be many different colors! 🍎"

### ChatPromptTemplate

ChatPromptTemplate는 대화목록을 프롬프트로 주입하고자 할 때 활용할 수 있습니다.

```python
ChatPromptTemplate.from_messages(
    messages: List[Union[Tuple[str, str], BaseMessagePromptTemplate]],
    input_variables: Optional[List[str]] = None,
    partial_variables: Optional[Dict[str, Any]] = None
)
```

- <code>messages</code>: 메시지들을 순서대로 나열한 리스트. 각 메시지는 ("role", "template string") 형태의 튜플이거나 SystemMessagePromptTemplate, HumanMessagePromptTemplate 등의 객체로 구성할 수 있음.
- <code>input_variables</code>: {}로 들어가는 변수명들의 리스트 (자동 추론되므로 대부분 생략 가능)
- <code>partial_variables</code>: 일부 변수를 미리 값 또는 함수로 지정 (예: 날짜, 사용자 ID 등 고정값 처리용)

**즉 ChatPromptTemplate란, 이전 대화들의 주체를 Role로서 구성하고 나눈 대화들을 content로서 넣어서 // 자연스러운 대화 결과를 얻기 위한 Prompt라고 생각할 수 있다.**


#### PromptTemplate vs. ChatPromptTemplate

| 항목          | `PromptTemplate`                              | `ChatPromptTemplate`                                     |
|---------------|-----------------------------------------------|-----------------------------------------------------------|
| **형식**       | 일반 텍스트 (문장 하나)                        | Chat 형태 (역할 기반 메시지 나열)                          |
| **역할 설정**  | ❌ 없음                                       | ✅ `system`, `user`, `assistant` 등의 역할 지정 가능         |
| **사용 모델**  | 일반 LLM (`text-davinci`, `llama`, 등)        | Chat LLM (`gpt-3.5-turbo`, `gpt-4`, `claude` 등)           |
| **메시지 순서**| 하나의 문자열                                 | 여러 메시지를 순서대로 입력 가능                            |
| **구성 요소**  | `{}`로 변수를 치환하는 하나의 문자열            | `SystemMessagePromptTemplate`, `HumanMessagePromptTemplate` 등 |



In [14]:
chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

chain = chat_template | model
chain.invoke({'name':'Teddy', 'user_input': '당신의 이름은 무엇입니까?'}).content.split('</think>\n\n')[1]

'안녕하세요! 저는 테디입니다. AI 어시스턴트입니다. 무엇을 도와드릴까요? 😊'

### MessagePlaceHolder

```python
MessagesPlaceholder(
    variable_name: str
)
```

MessagePlaceHolder는 특정 값을 넣을 때 사용된다. 보통 **이전 대화 기록**이나 **메모리 출력** 등을 넣는 데 사용된다. input_variables와 차이가 없어보이나 아래와 같은 차이가 있다.

#### input_variables vs. MessagePlaceHolder
| 항목 | 설명 |
|------|------|
| `input_variables` | `{}` 안에 들어갈 일반 문자열 값 (예: 숫자, 이름 등) |
| `MessagesPlaceholder` | 여러 개의 메시지 객체를 프롬프트 중간에 삽입할 때 사용 |

- `input_variables`는 문자열 템플릿을 채우는 용도
- `MessagesPlaceholder`는 Chat 구조 내에서 역할을 갖는 메시지들을 중간에 삽입할 수 있도록 함
- 보통 **이전 대화 기록**이나 **메모리 출력** 등을 넣는 데 사용됨


실제 MessagePlaceholder가 치환될 수 있는 값들은 아래와 같다. Input으로 들어가게 될때는 (role, message) 형태로 들어가게 된다.

| 클래스           | 역할 설명         | role 값       | 예시 |
|------------------|------------------|---------------|------|
| `HumanMessage`   | 사용자 입력 역할   | `"user"`       | `HumanMessage(content="안녕")` |
| `AIMessage`      | AI 응답 역할      | `"assistant"`  | `AIMessage(content="안녕하세요!")` |
| `SystemMessage`  | 시스템 지침 역할   | `"system"`     | `SystemMessage(content="너는 요약 AI야")` |

LLM의 Ouput을 다음 LLM의 Input으로 사용하고 싶을 때, PlaceHolder를 사용하는 법은 아래와 같다.

In [15]:
# Step 1: 첫 번째 프롬프트 (단일 질문)
first_prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 친절한 AI야."),
    ("human", "{question}"),
])

# 사용자 질문
user_question = "오늘 날씨 어때?"

# LLM에 첫 번째 질문 수행
first_result = model.invoke(first_prompt.format_messages(question=user_question))
print(first_result.content)

<think>
사용자의 질문은 "오늘 날씨 어때?"입니다. 이 질문은 날씨 정보를 제공하는 것이 필요합니다. 하지만 현재 시점에서 실제 날씨 데이터를 가져오는 기능이 없기 때문에, 대답을 할 수 없습니다. 대신, 사용자에게 예측 가능한 날씨 정보를 제공하거나, 날씨를 확인할 수 있는 방법을 제안할 수 있습니다. 예를 들어, "지역을 알려주시면 해당 지역의 날씨를 확인해 드릴 수 있어요."라고 말할 수 있습니다. 또한, 사용자에게 날씨를 확인하는 방법을 안내해줄 수도 있습니다. 예를 들어, "앱이나 웹사이트를 사용하여 실시간 날씨를 확인할 수 있어요."라고 말할 수 있습니다. 이와 같은 대답은 사용자를 도움을 주고, 사용자가 원하는 정보를 제공할 수 있도록 도와줍니다. 또한, 사용자에게 친절하게 대답하는 것이 중요합니다. 사용자가 질문을 하면, 즉시 대답하고, 필요하다면 추가적인 정보를 제공하는 것이 좋습니다. 따라서, 사용자에게 지역을 묻고, 날씨를 확인하는 방법을 제안하는 것이 적절합니다.
</think>

안녕하세요! 날씨 정보를 알려드릴 수 없어요. 현재 시점에서 실시간 날씨 데이터를 확인할 수 없기 때문입니다. 하지만 지역을 알려주시면, 해당 지역의 날씨를 확인해 드릴 수 있어요. 예를 들어, "서울", "부산", "경기" 등 어떤 지역이신가요? 또는 날씨를 확인할 수 있는 앱이나 웹사이트를 추천해드릴 수도 있어요! 😊


In [16]:
# Step 2: 생성된 응답을 기반으로 대화 기록 구성
first_chat_result = first_result.content.split('</think>\n\n')[1]

chat_history = [
    HumanMessage(content=user_question),
    AIMessage(content=first_chat_result)
]

# Step 3: 두 번째 프롬프트 - 이전 대화를 문맥으로 사용
second_prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 요약 전문 AI야."),
    MessagesPlaceholder(variable_name="history"),  # 이전 메시지 들어갈 자리
    ("human", "방금 대화를 한 문장으로 요약해줘."),
])

# history에 이전 대화 기록 전달
second_result = model.invoke(
    second_prompt.format_messages(
        history=chat_history
    )
)

print(second_result.content.split('</think>\n\n')[1])

우리의 대화는 사용자가 오늘 날씨를 묻고, AI가 실시간 날씨 정보를 제공할 수 없음을 설명하며 지역을 지정하거나 앱 추천을 제안하는 내용을 포함한 것입니다.


### FewShotPromptTemplate

```python
FewShotPromptTemplate(
    examples: List[dict],
    example_prompt: PromptTemplate,
    prefix: Optional[str] = None,
    suffix: Optional[str] = None,
    input_variables: List[str],
    example_separator: str = "\n\n"
)
```

| 인자명                 | 타입               | 설명                       |
| ------------------- | ---------------- | ------------------------ |
| `examples`          | `List[dict]`     | 예시 샘플 데이터 (입출력 쌍)        |
| `example_prompt`    | `PromptTemplate` | 각 예시를 어떻게 표현할지 정의        |
| `prefix`            | `str`            | 예시 앞에 붙는 도입 문구           |
| `suffix`            | `str`            | 실제 사용자 입력이 들어가는 부분       |
| `input_variables`   | `List[str]`      | `suffix`에서 사용되는 변수 목록    |
| `example_separator` | `str`            | 예시 사이의 구분자 (기본값: `\n\n`) |


FewShot이란 LLM에 실제 예시를 주어짐으로서, Output의 형태를 지정하거나 혹은 어떠한 형태를 원하는지 정보를 전달하여 할루시네이션을 줄일 수 있습니다. 실제 사용하는 예시는 아래와 같습니다.

In [17]:
# Fewshow에 사용할 Examples를 선언합니다. 해당 값은 List[dict]입니다.
examples = [
    {"english": "apple", "korean": "사과"},
    {"english": "car", "korean": "자동차"},
    {"english": "school", "korean": "학교"}
]

# 각 예시를 LLM에 넣을 Prompt Template입니다.
example_prompt = PromptTemplate(
    input_variables=["english", "korean"], # 변수로 받을 값 입니다. english, korean을 입력으로 받습니다.
    template="Q: {english}\nA: {korean}" # 입력받은 값으로서 어떻게 출력할지 형태를 정합니다.
)

print('Single Dataset Check')
print(example_prompt.format(**examples[0]))

# FewShotTemplate 선언
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="다음은 영어 단어를 한국어로 번역한 예시입니다:\n",
    suffix="Q: {english}\nA:",
    input_variables=["english"],
    example_separator="\n\n"
)

# 실제 입력 넣기
print("\n\nFewShot Prompt 결과 확인.")
formatted_prompt = few_shot_prompt.format(english="computer")
print(formatted_prompt)

Single Dataset Check
Q: apple
A: 사과


FewShot Prompt 결과 확인.
다음은 영어 단어를 한국어로 번역한 예시입니다:


Q: apple
A: 사과

Q: car
A: 자동차

Q: school
A: 학교

Q: computer
A:


In [18]:
# LLM에 사용할 Template 만들기
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 영어 단어를 한국어로 번역해주는 AI입니다."),
    ("human", formatted_prompt)
])

print(model.invoke(chat_prompt.format_messages()).content.split('</think>\n\n')[1])

Q: computer  
A: 컴퓨터


### Example Selector

- 참조 사이트: https://python.langchain.com/v0.1/docs/modules/model_io/prompts/example_selectors/

Fewshot의 예시를 많이 넣어두고, RAG에서 선택해서 필요한 Examples를 가져다 쓰게 하는게 보통 방법이다.  
다양한 방법이 있지만, 몇몇 방법만 예제로서 확인해보자.


#### Step1. Dataset 생성.

실제 Example Selector에서 사용할 Dataset을 생성한다.

In [19]:
examples = [
    {"english": "apple", "korean": "사과"},
    {"english": "car", "korean": "자동차"},
    {"english": "school", "korean": "학교"},
    {"english": "computer", "korean": "컴퓨터"},
    {"english": "house", "korean": "집"},
    {"english": "pencil", "korean": "연필"},
    {"english": "teacher", "korean": "선생님"},
    {"english": "student", "korean": "학생"},
    {"english": "city", "korean": "도시"},
    {"english": "book", "korean": "책"}
]

**Appendix: Text to Embedding**: 실제 Embedding Model을 사용하는 것이 가장 정확하나. Local환경이 3090 GPU 1개로는 qwen3-4b + embedding model을 같이 올릴 수 없다. 따라서, dummy로 embedding하는 function으로서 embedding 값을 사용하자.

In [20]:
'''
Dummy Embedding Function
1. text의 앞부분 5글자만 가져와서 float으로 변경
2. embed_documents, embed_query는 Embeddings에서 정의된 함수로서 내부적으로 싸져서 자동으로 사용할 수 있게 한다.
'''

class DummyEmbeddings(Embeddings):
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        # 실제로는 Qwen3 임베딩 사용 필요
        return [[float(ord(c)) for c in text][:5] for text in texts]

    def embed_query(self, text: str) -> List[float]:
        return [float(ord(c)) for c in text][:5]
    
embedding = DummyEmbeddings()
embedding.embed_documents(texts=['english'])

[[101.0, 110.0, 103.0, 108.0, 105.0]]

**추가 팁: 어떤값이 Embedding 되는 것 인가?**

DummyEmbedding을 보게 되면, document를 embedding하고, query를 embedding하는 것을 알 수 있다. 어떤 값을 기준으로 Embedding하는지 알아보면 다음과 같다.

1. input_keys가 지정되었으면 그 key만 추출
2. 지정되지 않았으면 dict.keys()[0] (첫 번째 key) 사용
3. 해당 문자열들을 .embed_documents()로 전달
4. 나중에 .select_examples(input) 호출 시, embed_query(input)과 벡터 유사도를 비교하여 가장 가까운 예시 k개 반환

#### Step2. Example Selector 종류

실제로 많이 사용되는 ExampleSelector를 살펴보자.

| Selector 클래스                          | 설명                                     |
| ------------------------------------- | -------------------------------------- |
| `SemanticSimilarityExampleSelector`   | 입력값과 임베딩 유사도가 높은 예시 `k`개 선택 (가장 많이 사용) |
| `LengthBasedExampleSelector`          | 예시 길이 기반으로 `max_length` 이내에서 가능한 만큼 선택 |
| `MaxMarginalRelevanceExampleSelector` | 유사성과 다양성을 동시에 고려하여 예시 선택               |


#### Step3. SemanticSimilarityExampleSelector

실제, ExampleSelector중 하나로서, 해당 결과를 확인해보자.

```python
SemanticSimilarityExampleSelector.from_examples(
    examples: List[dict],
    embedding: Embeddings,
    vectorstore_cls: Type[VectorStore],
    k: int = 4,
    input_keys: Optional[List[str]] = None
)
```
| 인자명               | 타입                | 설명                                               |
| ----------------- | ----------------- | ------------------------------------------------ |
| `examples`        | `List[dict]`      | 예시 입력/출력 쌍들 (예: {"english": ..., "korean": ...}) |
| `embedding`       | `Embeddings` 객체   | 예시의 입력을 벡터로 변환할 임베딩 모델                           |
| `vectorstore_cls` | `VectorStore` 클래스 | 벡터를 저장하고 유사도 검색할 벡터스토어 클래스 (예: `FAISS`)          |
| `k`               | `int`             | 입력값과 유사한 예시 몇 개를 선택할지 결정                         |
| `input_keys`      | `List[str]` (선택)  | 어떤 key의 값을 임베딩 대상으로 사용할지 지정 (기본은 첫 키)            |



In [21]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector

# Prompt Template으로서 english, korean을 Few Shot으로 넣기 위해 지정.
example_prompt = PromptTemplate(
    input_variables=["english", "korean"], # key값의 2개는 english, korean이다.
    template="Q: {english}\nA: {korean}" # 해당 값을 활용한 template는 다음과 같다.
)

# SemanticSearch로서 Embedding Vector를 구성한다.
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples, # input dataset
    embeddings=DummyEmbeddings(),  # 현재 GPU Memory 부족으로서 Dummy Embedding Function을 사용하여 Embedding 한다.
    vectorstore_cls=FAISS, # 저장하는 vector store는 FAISS로서 지정한다.
    k=2 # Few Shot의 갯수를 2개로 지정한다.
)

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="다음은 영어 단어를 한국어로 번역한 예시입니다:\n",
    suffix="Q: {english}\nA:",
    input_variables=["english"]
)

print(few_shot_prompt.format(english="keyboard"))

다음은 영어 단어를 한국어로 번역한 예시입니다:


Q: pencil
A: 연필

Q: student
A: 학생

Q: keyboard
A:


In [22]:
# LLM에 사용할 Template 만들기
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 영어 단어를 한국어로 번역해주는 AI입니다."),
    ("human", few_shot_prompt.format(english="keyboard"))
])

print(model.invoke(chat_prompt.format_messages()).content.split('</think>\n\n')[1])

Q: keyboard  
A: 키보드


**Appendix. multi_index for RAG**: 아래와 같이 여러 key값을 모두 embedding에 사용할 수 있다.

현재, DummyEmbedding에서는 2개 이상의 key값으로 Embedding하는 function을 지정하지 않아서, 사용하지는 못한다.

In [23]:
# SemanticSearch로서 Embedding Vector를 구성한다.
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples, # input dataset
    embeddings=DummyEmbeddings(),  # 현재 GPU Memory 부족으로서 Dummy Embedding Function을 사용하여 Embedding 한다.
    vectorstore_cls=FAISS, # 저장하는 vector store는 FAISS로서 지정한다.
    k=2, # Few Shot의 갯수를 2개로 지정한다.
    input_keys=["english", "korean"]
)

### LangChain-Hub

LangChain-Hub에 들어가게 되면, 아래 Figure와 같이, 남들이 사용한 Prompt를 확인하여 사용할 수 있다.
- link: https://smith.langchain.com/hub

![png](./img/1.png)

Prompt는 LangCahin-Hub에 연결하여, push하거나 pull하여 사용할 수 있다.

<hr>
참조: <a href="https://wikidocs.net/book/14314">LangChain 위키독스</a><br>
참조: <a href="https://github.com/wjddyd66/LLM/tree/main">원본 코드</a><br>

코드에 문제가 있거나 궁금한 점이 있으면 wjddyd66@naver.com으로  Mail을 남겨주세요.